<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 6. Redes Neuronales Recurrentes</font></h1>

<h1><font color="#113D68" size=4>6. Práctica: Procesamiento del Lenguaje Natural</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Descripción del problema: generación de texto](#section1)
* [2. LSTM de linea base](#section2)
    * [2.1. Cargar el dataset](#section2.1)
    * [2.2. Conversión a numérico](#section2.2)
    * [2.3. Dimensiones del dataset](#section2.3)
    * [2.4. Procesamiento de datos](#section2.4)
    * [2.5. Diseño de la LSTM](#section2.5)
    * [2.6. Crear puntos de control](#section2.6)
    * [2.7. Resultados](#section2.7)
* [3. Generación de texto con LSTM](#section3)
    * [3.1. Cargar los pesos de LSTM](#section3.1)
    * [3.2. Convertir de entero a carácter](#section3.2)
    * [3.3. Resultados y evaluación](#section3.3)
* [4. LSTM más profunda](#section4)
* [5. Mejorar nuestro modelo](#section5)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este proyecto, aprenderemos a crear un modelo generativo de texto utilizando LSTM:
* Descripción de un modelos generativos de texto.
* Enmarcar el problema de las secuencias de texto a un modelo generativo.
* Desarrollar una LSTM para generar secuencias de texto.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Descripción del problema: generación de texto</font>

VAmos a trabajar el libro de "Alicia en el país de las maravillas", por lo que podemos descargarlo de la página (Texto sin formato UTF-8) de este libro de forma gratuita y colocarlo en su directorio de trabajo con el nombre `wonderland.txt`. 

Abrimos el archivo y eliminamos el encabezado:
```
        *** START OF THIS PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***
```

El pie de página es todo el texto después de la línea de texto que dice:
```
        THE END
```

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el [Proyecto Gutenberg](https://www.gutenberg.org/)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Descargar el libro [Alice's Adventures in Wonderland](https://www.gutenberg.org/ebooks/11)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. LSTM de linea base</font>

En esta sección desarrollaremos una red LSTM simple para aprender secuencias de caracteres de Alicia en el país de las maravillas. 

<a id="section2.1"></a>
# <font color="#004D7F" size=5>2.1. Cargar el dataset</font>

Comencemos importando las clases y funciones que pretendemos usar para entrenar nuestro modelo.

Debemos cargar el texto ASCII y convertir todos los caracteres a minúsculas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text and covert to lowercase
file= "/content/drive/MyDrive/CursoDeepLearning/Datasets/wonderland.txt"
texto= open(file, 'r', encoding='utf-8').read()
texto= texto.lower()
texto

'\ufeffchapter i.\ndown the rabbit-hole\n\n\nalice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into\nthe book her sister was reading, but it had no pictures or\nconversations in it, “and what is the use of a book,” thought alice\n“without pictures or conversations?”\n\nso she was considering in her own mind (as well as she could, for the\nhot day made her feel very sleepy and stupid), whether the pleasure of\nmaking a daisy-chain would be worth the trouble of getting up and\npicking the daisies, when suddenly a white rabbit with pink eyes ran\nclose by her.\n\nthere was nothing so _very_ remarkable in that; nor did alice think it\nso _very_ much out of the way to hear the rabbit say to itself, “oh\ndear! oh dear! i shall be late!” (when she thought it over afterwards,\nit occurred to her that she ought to have wondered at this, but at the\ntime it all seemed quite natural); but when the rabbit actually

<a id="section2.2"></a>
# <font color="#004D7F" size=5>2.2. Conversión a numérico</font>

No podemos modelar los caracteres directamente, sino que debemos convertir los caracteres a números enteros. 

Por ejemplo, la lista de caracteres en minúscula ordenados únicos en el libro es la siguiente:

In [ ]:
# set(texto) -> diccionario
# list() -> to array
# sorted() -> ordenado

In [ ]:
def b(text,alist,newchar):
    for ch in alist:
        if ch in text:
            text = text.replace(ch,newchar)
    return text

texto= b(texto,['!','(',')','*',',','.',':',';','?','[',']','_','—','‘','’','“','”','-','\ufeff'],' ')
texto= b(texto,['ù',],'u')

In [ ]:
# create mapping of unique chars to integers
caracteres= sorted(list(set(texto))) 
char_to_int= dict((c,i) for i,c in enumerate(caracteres))
char_to_int

{'\n': 0,
 ' ': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

<a id="section2.3"></a>
# <font color="#004D7F" size=5>2.3. Dimensiones del dataset</font>

Ahora que se cargó el libro y se preparó el mapeo, podemos resumir el conjunto de datos.

In [ ]:
# summarize the loaded data
n_caracteres= len(texto)
n_vocabulario= len(caracteres) 

print('Longitud del libro:',n_caracteres)
print('Vocabulario:',n_vocabulario)

Longitud del libro: 144005
Vocabulario: 28


En este tutorial, dividiremos el texto del libro en subsecuencias con una longitud fija de 100 caracteres, una longitud arbitraria. 

Cada patrón de entrenamiento de la red se compone de 100 pasos de tiempo de un carácter (X) seguidos de una salida de carácter (y). 

Por ejemplo, si la longitud de la secuencia es 5 (para simplificar), los dos primeros patrones de entrenamiento serían los siguientes:
```
            CHAPT -> E
            HAPTE -> R
```
A medida que dividimos el libro en estas secuencias, convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente.

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_lon= 100
dataX= []
dataY= []
for i in range(0, n_caracteres-seq_lon, 1):
  seq_in= texto[i:i+seq_lon] 
  seq_out= texto[i+seq_lon]
  dataX.append([char_to_int[caracter] for caracter in seq_in])
  dataY.append(char_to_int[seq_out])

n_patrones= len(dataX)
print('Total Patterns:', n_patrones)

Total Patterns: 143905


<a id="section2.4"></a>
# <font color="#004D7F" size=5>2.4. Procesamiento de datos</font>

Ahora que hemos preparado nuestros datos de entrenamiento, necesitamos transformarlos. 
1. Transformar la lista de secuencias de entrada en la forma **[muestras, pasos de tiempo, características].** 
2. Cambiar la escala de los números enteros al rango de 0 a 1 y usar la función de activación sigmoidea.
3. Convertir los patrones de salida (caracteres individuales convertidos en enteros) con One-Hot Encoding. 

In [ ]:
# reshape X to be [samples, time steps, features]
X= np.reshape(dataX, (n_patrones,seq_lon,1))  

# normalize
X= X/float(n_vocabulario)

# one hot encode the output variable
y= np_utils.to_categorical(dataY)  
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
print(X.shape)
print(y.shape)

(143905, 100, 1)
(143905, 28)


<a id="section2.5"></a>
# <font color="#004D7F" size=5>2.5. Diseño de la LSTM</font>

Ahora podemos definir nuestro modelo LSTM. 
1. Definimos una única capa LSTM oculta con 256 unidades de memoria. 
2. La red utiliza un Dropout del 20%. 
3. La capa de salida es una capa densa que utiliza la función de activación Softmax. 
4. Compilación con pérdida logarítmica (`categorical_crossentropy`)
5. Usaremos el algoritmo de optimización de Adam.

In [ ]:
# define the LSTM model
model= Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 


<a id="section2.6"></a>
# <font color="#004D7F" size=5>2.6. Crear puntos de control</font>

Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección.

In [ ]:
# define the checkpoint
ruta= '/content/sample_data/pesos/pesos-mejora-{epoch:02d}-{loss:.4f}.hdf5' 
control= ModelCheckpoint(ruta, monitor='loss', verbose=1, save_best_only=True, mode='min')

callbacks_lista=[control]

<a id="section2.7"></a>
# <font color="#004D7F" size=5>2.7. Resultados</font>

Utilizamos un número modesto de 20 épocas y un gran tamaño de batch de 128 patrones.

In [ ]:
# fit the model
model.fit(X,y, epochs=20,batch_size=128, callbacks=callbacks_lista, verbose=2)  

Epoch 1/20
1125/1125 - 44s - loss: 2.7705 - accuracy: 0.2312

Epoch 00001: loss improved from inf to 2.77051, saving model to /content/sample_data/pesos/pesos-mejora-01-2.7705.hdf5
Epoch 2/20
1125/1125 - 37s - loss: 2.6379 - accuracy: 0.2558

Epoch 00002: loss improved from 2.77051 to 2.63788, saving model to /content/sample_data/pesos/pesos-mejora-02-2.6379.hdf5
Epoch 3/20
1125/1125 - 37s - loss: 2.5470 - accuracy: 0.2763

Epoch 00003: loss improved from 2.63788 to 2.54699, saving model to /content/sample_data/pesos/pesos-mejora-03-2.5470.hdf5
Epoch 4/20
1125/1125 - 37s - loss: 2.4604 - accuracy: 0.2963

Epoch 00004: loss improved from 2.54699 to 2.46043, saving model to /content/sample_data/pesos/pesos-mejora-04-2.4604.hdf5
Epoch 5/20
1125/1125 - 37s - loss: 2.4066 - accuracy: 0.3103

Epoch 00005: loss improved from 2.46043 to 2.40659, saving model to /content/sample_data/pesos/pesos-mejora-05-2.4066.hdf5
Epoch 6/20
1125/1125 - 37s - loss: 2.3464 - accuracy: 0.3280

Epoch 00006: loss

In [ ]:
score= model.evaluate(X,y, verbose=0)
print('Model Accuracy: %.2f%%' % (score[1]*100))   

Model Accuracy: 50.27%


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Generación de texto con LSTM</font>

La generación de texto utilizando la red LSTM entrenada es relativamente sencilla. 

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Cargar los pesos de LSTM</font>

En primer lugar, cargamos los datos y definimos la red exactamente de la misma manera

Los pesos de la red se cargan desde un archivo de punto de control y no es necesario entrenar la red. 

In [ ]:
# load the network weights
ruta_pesos= '/content/drive/MyDrive/CursoDeepLearning/Datasets/pesos-mejora-20-1.8357.hdf5'
model.load_weights(ruta_pesos)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 

<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Convertir de entero a carácter</font>

Creamos un mapeo inverso que podamos usar para convertir los números enteros de nuevo en caracteres.

In [ ]:
# summarize the loaded data
int_to_char=  dict((i,c) for i,c in enumerate(caracteres))
int_to_char

{0: '\n',
 1: ' ',
 2: 'a',
 3: 'b',
 4: 'c',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'h',
 10: 'i',
 11: 'j',
 12: 'k',
 13: 'l',
 14: 'm',
 15: 'n',
 16: 'o',
 17: 'p',
 18: 'q',
 19: 'r',
 20: 's',
 21: 't',
 22: 'u',
 23: 'v',
 24: 'w',
 25: 'x',
 26: 'y',
 27: 'z'}

<a id="section3.3"></a>
# <font color="#004D7F" size=5>3.3. Resultados y evaluación</font>

Finalmente, necesitamos realmente hacer predicciones de manera que:
1. Comenzamos primero con una semilla como entrada
2. Generamos el siguiente carácter y 
3. Actualizamos la semilla para agregar el carácter generado al final y recortar el primer carácter. 

Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1000 caracteres de longitud). 

In [ ]:
import sys

# pick a random seed
inicio= np.random.randint(0, len(dataX)-1) 
patron= dataX[inicio]

print('Semilla')
print('\"',''.join([int_to_char[valor] for valor in patron]),'\"')

for x in range(1000):
  x= np.reshape(patron, (1,len(patron),1))
  x= x/float(n_vocabulario)
  prediccion= model.predict(x,verbose=0)
  indice= np.argmax(prediccion)
  resultado= int_to_char[indice]
  seq_in= [int_to_char[valor] for valor in patron]
  sys.stdout.write(resultado)
  patron.append(indice)
  patron= patron[1:len(patron)]

Semilla
" for the duchess  an invitation from the
queen to play croquet   the frog footman repeated  in the sa "
me toie      lh the seie the had to soe king the rabeit  she was soierinng at the corr  she had not io   nd soee to  e teall      b  a i sgen  she had no tee rome   
  b dar ii the roeet   she said to herself   nh  oo  s all      io she taid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Podemos notar algunas observaciones sobre el texto generado.
* Generalmente se ajusta al formato de línea observado en el texto original de menos de 80 caracteres.
* Los caracteres están separados en grupos parecidos a palabras y la mayoría de los grupos son palabras reales en inglés (por ejemplo, _the, little_ y _was),_ pero muchos no (por ejemplo, _lott, tiie_ y _taede)._
* Algunas de las palabras en secuencia tienen sentido (por ejemplo, y el _white rabbit),_ pero otras muchas no (por ejemplo, _wese tilel)._

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. LSTM más profunda</font>

Ahora, podemos intentar mejorar la calidad del texto generado creando una red mucho más profunda. 

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from google.colab import drive
drive.mount('/content/drive')

# load ascii text and covert to lowercase
file= "/content/drive/MyDrive/CursoDeepLearning/Datasets/wonderland.txt"
texto= open(file, 'r', encoding='utf-8').read()
texto= texto.lower()

def b(text,alist,newchar):
    for ch in alist:
        if ch in text:
            text = text.replace(ch,newchar)
    return text

texto= b(texto,['!','(',')','*',',','.',':',';','?','[',']','_','—','‘','’','“','”','-','\ufeff'],' ')
texto= b(texto,['ù',],'u')

# create mapping of unique chars to integers
caracteres= sorted(list(set(texto))) 
char_to_int= dict((c,i) for i,c in enumerate(caracteres))
int_to_char=  dict((i,c) for i,c in enumerate(caracteres))

# summarize the loaded data
n_caracteres= len(texto)
n_vocabulario= len(caracteres) 

print('Longitud del libro:',n_caracteres)
print('Vocabulario:',n_vocabulario)

# prepare the dataset of input to output pairs encoded as integers
seq_lon= 100
dataX= []
dataY= []
for i in range(0, n_caracteres-seq_lon, 1):
  seq_in= texto[i:i+seq_lon] 
  seq_out= texto[i+seq_lon]
  dataX.append([char_to_int[caracter] for caracter in seq_in])
  dataY.append(char_to_int[seq_out])

n_patrones= len(dataX)
print('Total Patterns:', n_patrones)

# reshape X to be [samples, time steps, features]
X= np.reshape(dataX, (n_patrones,seq_lon,1))  

# normalize
X= X/float(n_vocabulario)

# one hot encode the output variable
y= np_utils.to_categorical(dataY)  

print(X.shape)
print(y.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Longitud del libro: 144005
Vocabulario: 28
Total Patterns: 143905
(143905, 100, 1)
(143905, 28)


In [6]:
# define the LSTM model
model= Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])  


También cambiaremos el nombre de archivo de los pesos con puntos de control para que podamos distinguir entre los pesos de esta red y la anterior (agregando la palabra más grande en el nombre del archivo).

In [9]:
# define the checkpoint
ruta= '/content/drive/MyDrive/CursoDeepLearning/Datasets/fit/pesos-mejora-{epoch:02d}-{loss:.4f}.hdf5' 
control= ModelCheckpoint(ruta, monitor='loss', verbose=1, save_best_only=True, mode='min')

callbacks_lista=[control] 


Finalmente, aumentaremos el número de épocas de entrenamiento de 20 a 50 y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender. 

In [10]:
# fit the model
model.fit(X,y, epochs=60,batch_size=64, callbacks=callbacks_lista)  


Epoch 1/60
2249/2249 [==============================] - 144s 64ms/step - loss: 2.1611 - accuracy: 0.3733

Epoch 00001: loss improved from inf to 2.16115, saving model to /content/drive/MyDrive/CursoDeepLearning/Datasets/fit/pesos-mejora-01-2.1611.hdf5
Epoch 2/60
2249/2249 [==============================] - 143s 64ms/step - loss: 1.9606 - accuracy: 0.4315

Epoch 00002: loss improved from 2.16115 to 1.96058, saving model to /content/drive/MyDrive/CursoDeepLearning/Datasets/fit/pesos-mejora-02-1.9606.hdf5
Epoch 3/60
2249/2249 [==============================] - 143s 64ms/step - loss: 1.8338 - accuracy: 0.4664

Epoch 00003: loss improved from 1.96058 to 1.83378, saving model to /content/drive/MyDrive/CursoDeepLearning/Datasets/fit/pesos-mejora-03-1.8338.hdf5
Epoch 4/60
2249/2249 [==============================] - 144s 64ms/step - loss: 1.7494 - accuracy: 0.4895

Epoch 00004: loss improved from 1.83378 to 1.74935, saving model to /content/drive/MyDrive/CursoDeepLearning/Datasets/fit/pesos-me

In [ ]:
#score= model.evaluate(X,y, verbose=0)
#print('Model Accuracy: %.2f%%' % (score[1]*100))   

Como en la sección anterior, podemos usar este mejor modelo de la ejecución para generar texto. 

El único cambio que debemos realizar es la especificación de la topología de la red y desde qué archivo se van a generar los pesos de la red. 

In [12]:
# load the network weights
ruta_pesos= '/content/drive/MyDrive/CursoDeepLearning/Datasets/pesos-mejora-46-1.0942.hdf5'
model.load_weights(ruta_pesos)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])  

import sys

# pick a random seed
inicio= np.random.randint(0, len(dataX)-1) 
patron= dataX[inicio]

print('Semilla')
print('\"',''.join([int_to_char[valor] for valor in patron]),'\"')

print('Texto generado:')
for x in range(1000):
  x= np.reshape(patron, (1,len(patron),1))
  x= x/float(n_vocabulario)
  prediccion= model.predict(x,verbose=0)
  indice= np.argmax(prediccion)
  resultado= int_to_char[indice]
  seq_in= [int_to_char[valor] for valor in patron]
  sys.stdout.write(resultado)
  patron.append(indice)
  patron= patron[1:len(patron)]


Semilla
" at all this time 

 i want a clean cup   interrupted the hatter   let s all move one place
on  

he  "
Texto generado:
said to the gryphon   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i m a poor man  your majesty   she said to herself   i

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section5"></a>
# <font color="#004D7F" size=6>5. Mejorar nuestro modelo</font>

A continuación, se muestra una muestra de ideas que tal vez desee investigar para mejorar aún más el modelo:
* Predecir menos de 1000 caracteres como salida para una semilla determinada.
* Eliminar toda la puntuación del texto fuente y, por tanto, del vocabulario de los modelos.
* Pruebe un One-Hot Encoding para las secuencias de entrada.
* Entrene al modelo en oraciones rellenas en lugar de secuencias aleatorias de caracteres.
* Aumentar el número de épocas de entrenamiento a 100 o más.
* Agregue Dropout a la capa de entrada visible y considere ajustar el porcentaje de Dropout.
* Ajuste el tamaño de batch, pruebe con un tamaño de batch de 1 como línea de base (muy lenta) y tamaños más grandes a partir de ahí.
* Agregue más unidades de memoria a las capas y / o más capas.
* Cambie las capas de LSTM para que tengan estado para mantener el estado en todos los batch.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>